In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

In [4]:
X1 = torch.rand(10000,1)
X2 = torch.rand(10000,1)
X3 = torch.rand(10000,1)
Y1 = ( (X1+X2+X3)<1 ).float()
Y2 = ( (1<(X1+X2+X3)) & ((X1+X2+X3)<2) ).float()
Y3 = ( (X1+X2+X3)>2 ).float()
Data = torch.cat([X1,X2,X3,Y1,Y2,Y3],axis=1) 
Data = Data.to('cuda:0') 
Data.shape

torch.Size([10000, 6])

In [5]:
train_size=int(0.7*len(Data))
test_size=len(Data)-train_size
Data=Data[torch.randperm(Data.size(0)),:]
train_data = Data[:train_size,:]
test_data = Data[train_size:,:]
train_data.shape,test_data.shape

(torch.Size([7000, 6]), torch.Size([3000, 6]))

In [6]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN,self).__init__()
        self.net = nn.Sequential(
            nn.Linear(3,5),nn.ReLU(),
            nn.Linear(5,5),nn.ReLU(),
            nn.Linear(5,5),nn.ReLU(),
            nn.Linear(5,3)
        )
    def forward(self,x):
        y=self.net(x)
        return y

In [7]:
model=DNN().to('cuda:0') 
model

DNN(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=5, bias=True)
    (5): ReLU()
    (6): Linear(in_features=5, out_features=3, bias=True)
  )
)

In [8]:
for name, param in model.named_parameters():
    print(f"参数:{name}\n 形状:{param.shape}\n 数值:{param}\n")

参数:net.0.weight
 形状:torch.Size([5, 3])
 数值:Parameter containing:
tensor([[-0.4040, -0.0127,  0.1977],
        [-0.2044, -0.2568,  0.5117],
        [ 0.4716, -0.5636, -0.1592],
        [-0.1127, -0.5541, -0.2132],
        [ 0.5099, -0.3308,  0.0813]], device='cuda:0', requires_grad=True)

参数:net.0.bias
 形状:torch.Size([5])
 数值:Parameter containing:
tensor([0.1696, 0.0758, 0.4064, 0.3594, 0.2713], device='cuda:0',
       requires_grad=True)

参数:net.2.weight
 形状:torch.Size([5, 5])
 数值:Parameter containing:
tensor([[-0.3373,  0.1262, -0.1479,  0.3877, -0.1810],
        [ 0.1351,  0.2927,  0.4272,  0.4231,  0.4033],
        [ 0.4465,  0.0707, -0.1494,  0.0428, -0.2486],
        [-0.0661, -0.3631, -0.0259,  0.3997,  0.0251],
        [-0.0792, -0.0081, -0.1655, -0.1497,  0.3647]], device='cuda:0',
       requires_grad=True)

参数:net.2.bias
 形状:torch.Size([5])
 数值:Parameter containing:
tensor([ 0.3205,  0.1441,  0.3399,  0.2114, -0.2892], device='cuda:0',
       requires_grad=True)

参数:net.4.wei

In [9]:
loss_fn = nn.MSELoss()

In [10]:
learning_rate = 0.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
epochs=1000
losses=[]
X=train_data[:,:3]
Y=train_data[:,-3:]
for epoch in range(epochs):
    Pred=model(X)
    loss=loss_fn(Pred,Y)
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
Fig=plt.figure()
plt.plot(range(epochs),losses)
plt.xlabel('epoch'),plt.ylabel('loss')
plt.show()